## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-09-20-51-12 +0000,bbc,Nepal anti-corruption protests explained,https://www.bbc.com/news/articles/crkj0lzlr3ro...
1,2025-09-09-20-51-01 +0000,bbc,Suspect in fatal stabbing of Ukrainian refugee...,https://www.bbc.com/news/articles/c4g7z8pk0j3o...
2,2025-09-09-20-50-07 +0000,nypost,Pair busted for swiping nearly $3M from LI cou...,https://nypost.com/2025/09/09/us-news/pair-swi...
3,2025-09-09-20-49-43 +0000,nypost,Supreme Court to weigh in on Trump’s emergency...,https://nypost.com/2025/09/09/us-news/supreme-...
4,2025-09-09-20-45-37 +0000,nyt,Supreme Court Agrees to Review Trump’s Sprawli...,https://www.nytimes.com/2025/09/09/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2302/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,39
292,new,26
0,nepal,16
204,israel,16
3,protests,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
32,2025-09-09-20-09-00 +0000,nypost,Trump was not on board with Israel’s strike in...,https://nypost.com/2025/09/09/us-news/trump-wa...,110
178,2025-09-09-13-31-30 +0000,cbc,Trump says Doha strike on Hamas 'does not adva...,https://www.cbc.ca/9.6895308?cmp=rss,101
229,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...,91
96,2025-09-09-17-44-10 +0000,nypost,White House fumes over Epstein birthday book s...,https://nypost.com/2025/09/09/us-news/wh-fumes...,89
26,2025-09-09-20-21-50 +0000,wapo,Live updates: Israeli strike targeting Hamas l...,https://www.washingtonpost.com/world/2025/09/0...,88


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
32,110,2025-09-09-20-09-00 +0000,nypost,Trump was not on board with Israel’s strike in...,https://nypost.com/2025/09/09/us-news/trump-wa...
81,83,2025-09-09-18-28-45 +0000,nypost,French President Emmanuel Macron names loyalis...,https://nypost.com/2025/09/09/world-news/frenc...
249,82,2025-09-09-07-15-58 +0000,wapo,"At least 19 killed, hundreds injured in protes...",https://www.washingtonpost.com/world/2025/09/0...
55,60,2025-09-09-19-29-47 +0000,nypost,Sicko accused of brutally stabbing Ukrainian r...,https://nypost.com/2025/09/09/us-news/ukrainia...
309,46,2025-09-08-22-53-00 +0000,nypost,Lachlan Murdoch takes control of media empire ...,https://nypost.com/2025/09/08/media/lachlan-mu...
252,45,2025-09-09-06-41-09 +0000,bbc,Thai court rules ex-PM Thaksin must serve one ...,https://www.bbc.com/news/articles/cly7k2g37g4o...
250,41,2025-09-09-07-10-42 +0000,nypost,Bill Clinton letter in Epstein ‘birthday book’...,https://nypost.com/2025/09/09/us-news/bill-cli...
196,40,2025-09-09-11-48-49 +0000,bbc,Israeli military orders all Gaza City resident...,https://www.bbc.com/news/articles/cvg47kvld8go...
111,39,2025-09-09-17-05-42 +0000,nypost,"US test scores hit damning new lows in math, r...",https://nypost.com/2025/09/09/us-news/us-test-...
147,36,2025-09-09-15-14-03 +0000,nypost,Iran has no path to producing weapons-grade ur...,https://nypost.com/2025/09/09/us-news/iran-has...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
